In [5]:
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/open.zip"

In [6]:
import os
import sys
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from PIL import ImageFile
from datetime import datetime
from tensorflow.keras.utils import to_categorical

sys.path.append(os.path.join(os.getcwd(), os.path.pardir, 'functions'))
from get_tensor_through_imgs_fn_v2 import get_tensor_through_imgs_fn

In [7]:
# ======== 학습 시킬 모든 이미지 파일들의 경로에 대한 리스트와 라벨 데이터 리스트 생성하기 ========

# c:\Users\UserK\Desktop\hansol_J\jiwon_work\..\..\open\train
TRAIN_FOLDER_PATH = os.path.join(os.getcwd(), os.pardir, os.pardir, 'open', 'train')
# TRAIN_FOLDER_PATH = '/content/train/'
# print('TRAIN_FOLDER_PATH : ', TRAIN_FOLDER_PATH)

def get_train_image_paths_and_labels_fn():

    defect_types = os.listdir(TRAIN_FOLDER_PATH)
    # print('defect_types :', defect_types)

    labels = []
    train_image_paths = []

    for defect_type in defect_types:

        file_names = os.listdir(os.path.join(TRAIN_FOLDER_PATH, defect_type))
        # print('file_names: ', file_names)

        for file_name in file_names:
            train_image_paths.append(os.path.join(TRAIN_FOLDER_PATH, defect_type, file_name))
            labels.append(defect_type)

    return train_image_paths, labels

train_image_paths, labels = get_train_image_paths_and_labels_fn()

# [ 하자 유형에 대한 폴더명\\파일명.확장자, ... ]
# print('train_image_paths: ', train_image_paths)

# [ 하자 유형에 대한 폴더명, ... ]
# print('labels :', labels)

# ==================================================================

In [8]:
# ========= 교수님 코드 따라서 이미지 데이터를 가져오고 차원 변형하기 =========

# 잘린 이미지에 대한 로드 여부 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

RESIZED_WIDTH = 227
RESIZED_HEIGHT = 227

train_image_tensor = get_tensor_through_imgs_fn(train_image_paths, RESIZED_WIDTH, RESIZED_HEIGHT)
print('train_image_tensor.shape :', train_image_tensor.shape)

# ==========================================================================


100%|██████████| 3457/3457 [01:19<00:00, 43.53it/s]


train_image_tensor.shape : (3457, 227, 227, 3)


In [9]:
# ========================== Alexnet 코드 따라서 학습시키기 ==========================
# https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98

EPOCHS = 50

# ====================== 라벨 정규화 과정 ======================

# 문자열 형태의 범주형 데이터인 라벨 데이터를 정수 형태로 정규화
normalized_labels = []

# 추후에 라벨이 추가될 경우 등 확장성 및 모델의 안정성을 고려하여 총 19가지의 하자 유형의 순서가 변경되지 않도록 직접적으로 리스트 생성
DEFECT_TYPE_NAMES = ["가구수정", "걸레받이수정", "곰팡이", "꼬임", "녹오염", "들뜸", "면불량", "몰딩수정", "반점", "석고수정",
                     "오염", "오타공", "울음", "이음부불량", "창틀,문틀수정", "터짐", "틈새과다", "피스", "훼손"]

defect_type_count = len(DEFECT_TYPE_NAMES)

for label in labels:
    normalized_labels.append(DEFECT_TYPE_NAMES.index(label))

# [ 0, 0, 0, ... ]
# print('normalized_labels :', normalized_labels)

# 정수형 클래스의 레이블을 이진 클래스의 원핫 인코딩 벡터로 변환
encoding_labels = to_categorical(normalized_labels, defect_type_count)
print('encoding_labels.shape :', encoding_labels.shape)

# =============================================================

train_ds = tf.data.Dataset.from_tensor_slices(
    (train_image_tensor, encoding_labels))

train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
print("Training data size:", train_ds_size)

train_ds = (train_ds
            .shuffle(buffer_size=train_ds_size)
            .batch(batch_size=32, drop_remainder=True))

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(
        4, 4), activation='relu', input_shape=(RESIZED_WIDTH, RESIZED_HEIGHT, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(
        5, 5), strides=(1, 1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(
        3, 3), strides=(1, 1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(
        3, 3), strides=(1, 1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(
        3, 3), strides=(1, 1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(19, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.SGD(learning_rate=0.001), metrics=['accuracy'])

model.summary()

model.fit(train_ds,
          epochs=EPOCHS)

# 연월일_시간
now = datetime.today().strftime('%Y%m%d_%H%M%S')
model.save(f'model_{now}.h5')

# ===================================================================================


encoding_labels.shape : (3457, 19)
Training data size: 3457


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 256)      1024      
 hNormalization)                                                 
                                                        